# 8 Queens

### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The EightQueensState class below, as well as the methods defined, should prove a helpful start for a Genetic Algorithms approach. However, you are welcome to change as little or as much of the code as is useful.

In [1]:
import numpy as np
from random import randint, choice


class EightQueensState:
    """This class represents a board in the eight queens puzzle"""

    def __init__(self, n, state=None, smarter_init_on=True):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            if smarter_init_on:
                state = self.smarter_init(n)
            else:
                state = np.random.randint(0, n, n)
        else:
            self.n = len(state)

        self.state = state
        self._fitness = ((self.n * (self.n - 1)) - self.cost()) / 2

    def smarter_init(self, queens):
        """This creates initial state by avoiding previously used column"""
        q1 = np.random.randint(0, queens, 1)[0]
        remaining = self.range_missing(0, queens, q1)
        state = np.empty(queens)
        state[0]=q1
        for i in range(1, queens):
            q1 = choice(remaining)
            state[i] = q1
            remaining.remove(q1)
        return state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
        return count

    def fitness(self):
        """Return the fitness valuation (n*(n-1)-cost)/2"""
        return self._fitness

    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def mutate(self):
        """Mutate a letter of the baby DNA and hope for the best"""
        new_state = self.state.copy()
        v = randint(0, self.n - 1)
        new_state[v] = randint(0, self.n - 1)
        return EightQueensState(self.n, state=new_state), v

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"


In [2]:
from random import random, randint, shuffle, choices
from EightQueensState import *


class EightQueensPopulation:
    """This class represents a generation of puzzle game states"""

    def __init__(self, queen_count, population=None, size=None, generation=0):
        """
        If no population is passed at init the population will be created based
        on the size parameter.

        :param state: pass in a list of EightQueensState objects
        """
        if not population:
            self._pop = []
            if size:
                for i in range(size):
                    self._pop.append(EightQueensState(queen_count))

        else:
            self._pop = population
        self._gen = generation
        self.n = queen_count
        self._log = []

    def print_state(self, dump_pop=False, dump_log=False):
        """
        Prints debug output

        :param dump_pop: prints the whole population
        :param dump_log: prints the step log including parents, mutations
        """
        avg_fit = sum([i.fitness() for i in self._pop]) / len(self._pop)
        probs = self.probabilities()
        print("Generation {}, avg fit {}".format(self._gen, avg_fit))
        if dump_pop:
            for i in range(len(self._pop)):
                print("{}, {}".format(self._pop[i], probs[i] * 100))
        if dump_log:
            for t in self._log:
                if len(t) == 4:
                    print("{} + {} at {} = {}".format(t[0], t[1], t[2], t[3]))
                elif len(t) == 1:
                    print("Mutated to {}".format(t[0]))

    def grow(self, i):
        """Adds DNA to the population"""
        self._pop.append(i)

    def size(self):
        """Returns the size of population"""
        return len(self._pop)

    @staticmethod
    def norm(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))

    def max_fitness(self):
        """Returns the maximum fitness score"""
        return (self.n * (self.n - 1)) / 2

    def probabilities(self):        
        return [i.fitness() / self.max_fitness() for i in self._pop]

    def cull(self, candidates, cutoff=0.2):
        """Cull worst solutions out of population"""
        return [(p, i) for p, i in candidates if p >= cutoff]

    def pick_n(self, n=2):
        """Random weighted pick from the population"""
        return choices(self._pop, weights=[i.fitness() for i in self._pop], k=n)

    def log(self, t):
        self._log.append(t)

    def offspring_V1(self, parents):
        """Create a new DNA by merging two parents' DNA codes at random point"""
        n = len(parents[0].state)
        inx = randint(0, n)
        left = parents[0].state.copy()[0:inx]
        right = parents[1].state.copy()[inx:n]
        return EightQueensState(self.n, state=np.concatenate([left, right])), inx

    def offspring_V2(self, parents):
        """Create a new DNA by merging two parents' DNA codes at random point.
           Take the better child"""
        n = len(parents[0].state)

        bestbaby = None
        bestinx = 0
        for tries in range(1):
            inx = randint(0, n)
            # baby 1
            left1 = parents[0].state.copy()[0:inx]
            right1 = parents[1].state.copy()[inx:n]
            b1 = EightQueensState(self.n, state=np.concatenate([left1, right1]))
            # baby 2
            left2 = parents[1].state.copy()[0:inx]
            right2 = parents[0].state.copy()[inx:n]
            b2 = EightQueensState(self.n, state=np.concatenate([left2, right2]))
            # return the better baby
            bestinx=inx
            if b1.fitness() <= b2.fitness():
                if not bestbaby or b2.fitness() > bestbaby.fitness():
                    bestbaby = b2
            else:
                if not bestbaby or b1.fitness() > bestbaby.fitness():
                    bestbaby = b1

        return bestbaby, bestinx

    def culling(self, percent=0.8, elite=None):
        """Remove X% of the least fit from the list based on fitness score"""
        l = sorted(self._pop, key=lambda x: x.fitness(), reverse=True)
        take = int(percent*len(l))
        l = l[:take]
        if elite:
            l.extend(elite)
        shuffle(l)
        self._pop = l

    def get_elite(self, percent=0.05):
        """Return top X% individuals based on the fitness score"""
        l = sorted(self._pop, key=lambda x: x.fitness(), reverse=True)
        take = int(percent*len(l))
        return l[:take]



In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
from random import random, randint, shuffle
from EightQueensPopulation import *

N = 8                # number of queens
N_POPULATION = 1000  # size of population
P_MUTATION = 0.1     # probability of mutation

def addstats(data, val):
    """Helper function used to collect statistics info"""
    if not data is None:
        data.append([i.fitness() for i in val._pop])

def genetic_algo(stdata=None, culling_on=True, elitism_on=True):
    """
    Genetic algorithm solving N queens
    :param culling_on: cull population each generation
    :param elitism_on: pass top 5% of population forwards
    """
    gen_i = 0  # Generation count
    old_gen = EightQueensPopulation(N, size=N_POPULATION, generation=gen_i)

    not_found = True
    while not_found:
        addstats(stdata, old_gen)
        next_gen = EightQueensPopulation(N, generation=gen_i + 1)

        for i in range(N_POPULATION):

            parents = old_gen.pick_n()
            baby, inx = next_gen.offspring_V1(parents)            

            if random() < P_MUTATION:
                baby, inx = baby.mutate()

            if (next_gen.size() < N_POPULATION):
                next_gen.grow(baby)

            not_found = not baby.is_goal()

            if not not_found:
                # Found solution:
                print(baby)
                break
                
        # If elitism is on, take top N% of individuals
        e = None
        if elitism_on:
            e = old_gen.get_elite()

        old_gen = next_gen

        if culling_on:
            old_gen.culling(elite=e)
        gen_i += 1

def vis_stats(stdata):
    """This is a visualization function to plot the progression of fitness scores"""
    print("Number of generations: ", len(stdata))
    sns.set_theme(style="whitegrid")

    x = []
    y1 = []
    y2 = []
    y3 = []

    for i in range(len(stdata)):
        l = stdata[i]
        x.append(i)
        y1.append(min(l))
        y2.append(sum(l) / len(l))
        y3.append(max(l))

    d = {'x': np.array(x), 'y1': np.array(y1), 'y2': np.array(y2), 'y3': np.array(y3)}
    df = pd.DataFrame(d)


    g = sns.FacetGrid(df["x"], height=8, aspect=2)
    g.map(sns.lineplot, x=df["x"], y=df["y1"], color="lightblue")
    g.map(sns.lineplot, x=df["x"], y=df["y2"], color='red')
    g.map(sns.lineplot, x=df["x"], y=df["y3"], color='lightgreen')
    plt.show()

def stats():
    """Runs a test 10 times and collects statistics, plots the progression of fitness scores"""
    Nruns = 10 # number of runs
    print("Running a test of {}".format(Nruns))
    print("Queens {}".format(N))
    print("Population {}".format(N_POPULATION))
    print("P mutation {}".format(P_MUTATION))

    results = {}
    for i in range(Nruns):
        print("Run ", i)
        statsdata = []
        start = time.time()
        genetic_algo(stdata=statsdata)
        end = time.time()
        print("Done in {} seconds ".format(end-start))
        results[i] = statsdata

    for key, s in results.items():
        vis_stats(s)

# Uncomment bottom line to run a test 10x and visualize results:
# stats()

genetic_algo()

Goal state! [1. 4. 6. 3. 0. 7. 5. 2.]
